In [2]:
import pandas as pd
import PyPDF2
import fitz
from io import StringIO

In [3]:
# This script is to read all characthers on a specific page on a PDF and Export into an Excel File

pdf_file = open('POL140298453100-lastpage_annots.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

text = ''
for page in range(len(pdf_reader.pages)):
    text += pdf_reader.pages[0].extract_text()

df = pd.read_csv(StringIO(text), sep='\t', header=None)

df.to_excel('output.xlsx', index=False, header=False)

In [5]:
import PyPDF2

# Open the PDF file
with open('POL140298453100-lastpage_annots.pdf', 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)

    # Get the first page of the PDF
    page = pdf_reader.pages[0]

    # Check if the page has any annotations
    if '/Annots' in page:
        # Loop through the annotations
        for annotation in page['/Annots']:
            # Check if the annotation is a rectangle
            if annotation['/Subtype'] == '/Square':
                print('Found an annotation rectangle!')
                break
    else:
        print('No annotations found.')


TypeError: 'IndirectObject' object is not subscriptable

In [3]:
doc = fitz.open('document_with_annotations.pdf')
pageone = doc[0]
words = pageone.get_text("words")

In [4]:
def make_text(words):
    line_dict = {} 
    words.sort(key=lambda w: w[0])
    for w in words:  
        y1 = round(w[3], 1)  
        word = w[4] 
        line = line_dict.get(y1, [])  
        line.append(word)  
        line_dict[y1] = line  
    lines = list(line_dict.items())
    lines.sort()  
    return "n".join([" ".join(line[1]) for line in lines])

In [4]:
rec = pageone.first_annot.rect

#Information of words in first object is stored in mywords

first_annots = []
mywords = [w for w in words if fitz.Rect(w[:4]) in rec]
ann = make_text(mywords)
first_annots.append(ann)

AttributeError: 'NoneType' object has no attribute 'rect'

In [9]:
print(rec)
print(first_annots)

Rect(76.98999786376953, 155.42999267578125, 223.1999969482422, 172.25)
['LETICIA HERMINIA MUÑOZ PEÑA']


In [5]:
all_annots = []

In [6]:
for annot in pageone.annots():
    if annot!=None:
        rec=annot.rect
        mywords = [w for w in words if fitz.Rect(w[:4]) in rec]
        ann= make_text(mywords)
        all_annots.append(ann)

In [7]:
print(all_annots)

['RAMON SANCHEZ GARAY', 'SAGR431217XXX', '150203067000', 'Contado']


In [8]:
pol_dic = {'NO.POLIZA': [all_annots[2]],
           'NOMBRE': [all_annots[0]], 
           'RFC': [all_annots[1]], 
           'FORMA PAGO': [all_annots[3]]}

In [9]:
pol_dic

{'NO.POLIZA': ['150203067000'],
 'NOMBRE': ['RAMON SANCHEZ GARAY'],
 'RFC': ['SAGR431217XXX'],
 'FORMA PAGO': ['Contado']}

In [10]:
pol_data = pd.DataFrame.from_dict(pol_dic)
pol_data


,NO.POLIZA,NOMBRE,RFC,FORMA PAGO
0,150203067000,RAMON SANCHEZ GARAY,SAGR431217XXX,Contado


In [11]:
pol_data.to_excel('pol_final.xlsx', index=False)

PermissionError: [Errno 13] Permission denied: 'pol_final.xlsx'